<a href="https://colab.research.google.com/github/geekpradd/Reinforcement-Learning-Stock-Trader/blob/master/Pendulum-DDPG/Untitled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Deep Deterministic Policy Gradient for Continuous Pendulum Control 

In [0]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import Input

In [0]:
from collections import deque
import random
class ReplayBuffer:
    def __init__(self, buffer_size):
        self.size = buffer_size
        self.buffer = deque()
        
    def add(self, data_tuple):
        self.buffer.append(data_tuple)
        if len(self.buffer) > self.size:
            self.buffer.poplef()
            
    def sample(self, sample_size):
        return random.sample(self.buffer, min(len(self.buffer), sample_size))

In [0]:
class Actor:
    def __init__(self, params):
        self.output_range = params["output_range"]
        self.hidden_layers = params["hidden_layers"]
        self.input_shape = params["input_shape"]
        self.actor = self.model(self.input_shape)
        
    def model(self, input_shape):
        input_layer =  Input(shape = input_shape)
        x = BatchNormalization()(input_layer)
        for layer in self.hidden_layers:
            x = Dense(layer, activation='relu')(x)
            x = BatchNormalization()(x)
        x = Dense(1, activation='tanh')(x)
        x = Lambda(lambda x: x*self.output_range)(x)
        model = tf.keras.Model(inputs=input_layer, outputs=x)
        return model
    
    def get_action(self, state):
        return self.actor.predict(np.array([state]))[0]
    
class Critic:
    def __init__(self, params):
        self.hidden_layers = params["hidden_layers"]
        self.input_shape = params["input_shape"]
        self.critic_online = self.model()
        self.critic_target = self.model()
    def model(self):
        input_layer =  Input(shape = input_shape)
        x = BatchNormalization()(input_layer)
        for layer in self.hidden_layers:
            x = Dense(layer, activation='relu')(x)
            x = BatchNormalization()(x)
        x = Dense(1, activation='linear')(x)
        model = tf.keras.Model(inputs=input_layer, outputs=x)
        return model
    
    def get_qvalues(self, state):
        return self.critic.predict(np.array([state]))[0]
    
    def merge_networks(self, tau):
        self.critic_target.set_weights(tau*np.array(self.critic_online.get_weights())
                                                                    + (1-tau)*np.array(self.critic_target.get_weights()))
        
    

In [0]:
class Agent:
    def __init__(self):
        self.actor = Actor(params["actor"])
        self.critic = Critic(params["critic"])
        self.buffer = ReplayBuffer(params["buffer_size"])
        self.epsilon = 1
        self.epsilon_decay = params["decay"]
        self.epsilon_min = params["min_epsilon"]
        self.discount = params["discount"]
        self.merge_frequency = params["merge_frequency"]
        self.save_frequency = params["save_frequency"]
        self.batch_size = params["batch_size"]
        self.optimizer = params["optimizer"]
        self.game = 0

In [0]:
inp = Input(shape = (4, ))

In [0]:
x = Dense(1, activation='relu')(inp)

In [0]:
model = tf.keras.Model(inputs=inp, outputs=x)
model.compile(loss='mse')

In [0]:
model.summary()
inpu = np.array([[1, 2, 3, 4]])
model.predict([inpu])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


InternalError:  Blas GEMV launch failed:  m=4, n=1
	 [[node model/dense/MatMul (defined at <ipython-input-5-0aa7b31ccf2a>:3) ]] [Op:__inference_predict_function_110]

Function call stack:
predict_function


In [0]:
with tf.GradientTape() as g:
    